# Checking schedules

This notebook contains:
1. Checking that the schedules submitted correctly via different flows (workflow, cli and run_function).
2. The access key of the schedule is not exposed.
3. Overwriting workflow scheduling works.

In [1]:
%config Completer.use_jedi = False

In [2]:
import mlrun
import os

In [3]:
run_db = mlrun.get_run_db()

## Loading project with workflows

In [4]:
PROJECT_NAME = "schedule-test"

In [5]:
project = mlrun.load_project(f"./{PROJECT_NAME}", "git://github.com/mlrun/project-demo.git", PROJECT_NAME)

In [6]:
def assert_schedule_access_key_is_not_exposed(name: str):
    schedule = run_db.api_call("GET", f"projects/{project.name}/schedules/{name}", params={"include-credentials": True}).json()
    assert schedule["credentials"]["access_key"].startswith("$ref:mlrun-auth-secrets.")
    print("Pass!")

def assert_schedule_overwritten(name: str, schedule: str):
    scheduled_job = run_db.get_schedule(project.name, name)
    assert scheduled_job.scheduled_object["schedule"] == schedule
    print("Pass!")
    

## Running the workflow

In [7]:
out = os.system(f"cd ./{PROJECT_NAME}; git config --global --add safe.directory {os.path.abspath(PROJECT_NAME)}")

In [8]:
project.run("main", schedule="*/10 * * * *")

> 2022-12-12 09:06:56,735 [warning] WARNING!, you seem to have uncommitted git changes, use .push()
> 2022-12-12 09:06:57,333 [info] executing workflow scheduling 'workflow-runner-main' remotely with kfp engine
> 2022-12-12 09:06:57,337 [info] starting run main uid=c4d4debef8fa4f2888bf4aa4bf0a771e DB=http://mlrun-api:8080
> 2022-12-12 09:06:57,629 [info] task scheduled, {'schedule': '*/10 * * * *', 'project': 'schedule-test', 'name': 'main'}


In [9]:
assert_schedule_access_key_is_not_exposed("main")

Pass!


## Overwriting a schedule 

In [10]:
project.run("main", schedule="*/20 * * * *", overwrite=True, dirty=True)

> 2022-12-12 09:06:57,778 [warning] WARNING!, you seem to have uncommitted git changes, use .push()
> 2022-12-12 09:06:57,819 [info] Deleting schedule main
> 2022-12-12 09:06:57,883 [info] executing workflow scheduling 'workflow-runner-main' remotely with kfp engine
> 2022-12-12 09:06:57,885 [info] starting run main uid=8333c1cf9538476c8f956d054113b09b DB=http://mlrun-api:8080
> 2022-12-12 09:06:58,130 [info] task scheduled, {'schedule': '*/20 * * * *', 'project': 'schedule-test', 'name': 'main'}


In [11]:
assert_schedule_access_key_is_not_exposed("main")
assert_schedule_overwritten("main", "*/20 * * * *")

Pass!
Pass!


### Check from CLI

In [12]:
out = os.popen(f"mlrun project ./{PROJECT_NAME} -r main -d --schedule '*/15 * * * *' -os").read()
print(out)

Loading project schedule-test into ./schedule-test:

kind: project
metadata:
  name: schedule-test
spec:
  description: test
  params:
    label_column: label
  functions:
  - url: prep_data.py
    name: prep-data
    image: mlrun/mlrun
    handler: prep_data
    with_repo: true
  - url: hub://sklearn_classifier
    name: train
  - url: hub://test_classifier
    name: test
  - url: hub://v2_model_server
    name: serve
  - url: gen_iris.py
    name: gen-iris
    image: mlrun/mlrun
    handler: iris_generator
    requirements:
    - requests
  workflows:
  - engine: kfp
    path: ./kflow.py
    name: main
  - path: ./newflow.py
    name: newflow
    handler: newpipe
  artifacts:
  - kind: ''
    target_path: https://s3.wasabisys.com/iguazio/data/iris/iris.data.raw.csv
    key: data
  source: git://github.com/mlrun/project-demo.git#refs/heads/main
  origin_url: git://github.com/mlrun/project-demo.git#refs/heads/main
  load_source_on_run: true
  desired_state: online

running workflow mai

In [13]:
assert_schedule_access_key_is_not_exposed("main")
assert_schedule_overwritten("main", "*/15 * * * *")

Pass!
Pass!


## Check function scheduling

In [14]:
mlrun.run_function("prep-data", schedule="*/10 * * * *", name="data-preparation")

> 2022-12-12 09:07:07,346 [info] starting run data-preparation uid=47dd2874dfe74fa98447ccb41c3979ad DB=http://mlrun-api:8080
> 2022-12-12 09:07:07,580 [info] task scheduled, {'schedule': '*/10 * * * *', 'project': 'schedule-test', 'name': 'data-preparation'}


In [15]:
assert_schedule_access_key_is_not_exposed("data-preparation")

Pass!
